# 探究性数据分析

In [3]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

init_notebook_mode(connected=True)

In [5]:
import numpy as np 
import pandas as pd 
df = pd.read_csv(r"C:/ML/project/chem/train.csv")
test = pd.read_csv(r"C:/ML/project/chem/test.csv")

features = list(set(df.columns.tolist()) - set(['formation_energy_ev_natom', 'bandgap_energy_ev', 'id']))
targets = ['formation_energy_ev_natom', 'bandgap_energy_ev']


### 皮尔逊相关系数

In [7]:
corr_df = df[features].corr()

data1 = [go.Heatmap(x=features,
                    y=features,
                    z=corr_df.values,
                   showscale = True,
                   zmin=-1, zmax=1)]

layout = dict(title="correlation heatmap- Features", 
                xaxis=dict(
        title='Features',
        titlefont=dict(
            size=18,
        ),
        showticklabels=False,
        ticks="",
        tickangle=90,
        tickfont=dict(
            size=4,
        ),
                
    ),
    yaxis=dict(
        title='Features',
        titlefont=dict(
            size=18,
        ),
        showticklabels=False,
        ticks="",
        tickangle=0,
        tickfont=dict(
            size=0,
        ),
    ),
    width = 750, height = 750,
    autosize = False )

figure = dict(data=data1,layout=layout)
iplot(figure)

In [8]:
corr = df[list(set(features)|set(['formation_energy_ev_natom','bandgap_energy_ev']))].corr()

data2 = [go.Heatmap(x=features,
                   y=targets,
                   z=corr.loc[targets, features].values)]

layout2 = dict(title="Correlation heatmap", 
                xaxis=dict(
        title='Features',
        titlefont=dict(
            size=18,
        ),
        showticklabels=False,
        tickangle=90,
        ticks="",
        tickfont=dict(
            size=4,
        ),
                
    ),
    yaxis=dict(
        title='Targets',
        titlefont=dict(
            size=18,
        ),
        showticklabels=False,
        tickangle=0,
        ticks="",
        tickfont=dict(
            size=4,
        ),
    ),
    width = 750, height = 275,
    autosize = False )

figure2 = dict(data=data2,layout=layout2)
iplot(figure2)

研究特征与特征，特征与目标之间的相关度，对于我们了解数据的特征，进行特征选择有很大的帮助，一般来说，我们会尽量在相关度大的特征之间进行取舍，剔除对目标影响较小的那个，或者将相关度较高的特征进行改造结合，而对于与目标相关度较高的特征，我们会更加关注它，试验它。
通过上图，我们可以发现：
1.Lattice_vector_1与lattice_angle_beta_degree高度相关，我们很有可能从它们两者的结合探索出新的特征。根据特征与目标的相关度图，我们选择与目标相关性最高的特征入手分析，通过交叉验证的方法来试验在加入或删除这些特征的前后，对于最终得到的结果有什么影响，从而有利于特征选择。
2.带隙能量与原子百分数高度相关，所以在之后的分子中会着重关注这个变量对于目标的影响。
3.形成能量与lattice_vector_3_ang高度相关，但与其他特征相关性较弱，在之后的分析中也会着重探究这个特征的影响。

### 晶格矢量与带隙能量的关系

In [10]:
x = df['lattice_vector_1_ang'].values
y = df['lattice_vector_2_ang'].values
z = df['lattice_vector_3_ang'].values


trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=df['bandgap_energy_ev'].values,                
        colorscale='Jet',   
        opacity=0.7
    )
)

data = [trace1]
layout = go.Layout(
    showlegend=True,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### 形成能与晶格矢量的关系

In [11]:
x = df['lattice_vector_1_ang'].values
y = df['lattice_vector_2_ang'].values
z = df['lattice_vector_3_ang'].values


trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=df['formation_energy_ev_natom'].values,               
        colorscale='Jet',   
        opacity=0.5
    )
)

data = [trace1]
layout = go.Layout(
    showlegend=True,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

可以看到，晶格矢量与目标之间并没有特别明显的相关性，因而在之后的模型中，我们需要注意两者在模型的预测结果中是否有明显的正负相关性，如果有，有必要对结果进行进一步的验证。

### 原子百分数与带隙能量的关系

为了解Al，In和Ga的这些原子百分比如何与彼此和目标相互作用，我们使用三元图将其可视化。 三元图很适合于化学成分可视化，因为它允许将三个变量绘制在2D图中，从而使得目标方便地通过热能图在三角形图上可视化。

In [13]:
def makeAxis(title, tickangle): 
    return {
      'title': title,
      'titlefont': { 'size': 20 },
      'tickangle': tickangle,
      'tickfont': { 'size': 15 },
      'tickcolor': 'rgba(0,0,0,0)',
      'ticklen': 5,
      'showline': True,
      'showgrid': True
    }


data = [{ 
    'type': 'scatterternary',
    'mode': 'markers',
    'a': df['percent_atom_al'].values,
    'b': df['percent_atom_in'].values,
    'c': df['percent_atom_ga'].values,
#     'hoverinfo': 'text',
#     'hovertext': '',
    'text': "a: AL, b: IN, c: GA",
    'marker': {
        'symbol': 100,
        'size': 12,
        'color': df['bandgap_energy_ev'].values,                # set color to an array/list of desired values
        'colorscale':'Jet',   # choose a colorscale
        'opacity': 0.8,
        'line': { 'width': 2.5 },
        'showscale': True
    },
    }]

layout = {
    'ternary': {
        'sum': 1,
        'aaxis': makeAxis('AL at. %', 0),
        'baxis': makeAxis('<br>IN at. %', 45),
        'caxis': makeAxis('<br>GA at. %', -45),
    },
    'annotations': [{
      'showarrow': False,
      'text': 'Bandgap Energy at different (AL, IN, GA) at. %',
        'x': 0.5,
        'y': 1.3,
        'font': { 'size': 15 }
    }]
}

fig = {'data': data, 'layout': layout}
iplot(fig, validate=False)

### 原子百分数与形成能的关系

In [14]:
data = [{ 
    'type': 'scatterternary',
    'mode': 'markers',
    'a': df['percent_atom_al'].values,
    'b': df['percent_atom_in'].values,
    'c': df['percent_atom_ga'].values,
    'hoverinfo': "a+b+c+name+text",
    'hovertext': "A: AL, B: IN, C: GA",
    'text': df['formation_energy_ev_natom'].values,
    'name': "",
    #'text': "a: AL, b: IN, c: GA",
    'marker': {
        'symbol': 100,
        'size': 12,
        'color': df['formation_energy_ev_natom'].values,                # set color to an array/list of desired values
        'colorscale':'Jet',   # choose a colorscale
        'opacity': 0.8,
        'line': { 'width': 2.5 },
        'showscale': True,
        'label': 'formation_energy_ev_natom'
    },
    }]

layout = {
    'ternary': {
        'sum': 1,
        'aaxis': makeAxis('AL at. %', 0),
        'baxis': makeAxis('<br>IN at. %', 45),
        'caxis': makeAxis('<br>GA at. %', -45),
    },
    'annotations': [{
      'showarrow': False,
      'text': 'Formation Energy at different (AL, IN, GA) at. % ',
        'x': 0.5,
        'y': 1.3,
        'font': { 'size': 15 }
    }]
}

fig = {'data': data, 'layout': layout}
iplot(fig, validate=False)

从上面的可视化结果我们可以看出：
1.在边缘，In含量几乎为0%的区域有着较小的形成能。
2.相似的现象对于Al也成立。
3.在Ga含量较小的区域，形成能有逐渐增加的趋势。
4.从In，Al含量较小的区域到Ga含量较小的的区域过程中，形成能有一个逐渐减小的趋势。

### 原子总数与形成能的关系

In [16]:
data = []
for number_of_total_atoms in df['number_of_total_atoms'].value_counts().index.tolist():
    y0 = df[df['number_of_total_atoms']==number_of_total_atoms]['formation_energy_ev_natom'].values
    data.append(go.Box(y=y0, name=str(number_of_total_atoms), boxpoints = 'suspectedoutliers',boxmean='sd'))
    
    layout = go.Layout(
        title = "Number of total atoms vs. Formation energy",
        yaxis=dict( title = 'Formation energy'),
        xaxis=dict( title = 'Number of total Atoms'))
    
iplot(go.Figure(data=data,layout=layout))

从总原子数与形成能的盒形图中可以看出，对于原子数为10的数据，其中很有可能存在异常值，需要进一步分析该分组的数据

### 原子总数与带隙能量的关系

In [17]:
data = []
for number_of_total_atoms in df['number_of_total_atoms'].value_counts().index.tolist():
    y0 = df[df['number_of_total_atoms']==number_of_total_atoms]['bandgap_energy_ev'].values
    data.append(go.Box(y=y0, name=str(number_of_total_atoms), boxpoints = 'suspectedoutliers',boxmean='sd'))
    layout = go.Layout(
        title = "Number of total atoms vs. Bandgap Energy",
        yaxis=dict( title = 'Bandgap Energy'),
        xaxis=dict( title = 'Number of total Atoms'))
iplot(go.Figure(data=data,layout=layout))

从上面的图中没有观察到明显的规律性。 然而，我们可以观察到对于不同的原子数量，Band-gap Energy和Formation Energies的变化似乎非常相似。这与总原子数和带隙能量与形成能量之间相关系数分别为0.158与0.142的结论相一致。

### 空间群与带隙能量的关系

In [18]:
data = []
for spacegroup in df['spacegroup'].value_counts().index.tolist():
    y0 = df[df['spacegroup']==spacegroup]['formation_energy_ev_natom'].values
    data.append(go.Box(y=y0, name=str(spacegroup), boxpoints = 'suspectedoutliers',boxmean='sd'))
    layout = go.Layout(
        title = "Spacegroup vs. Formation Energy",
        yaxis=dict( title = 'Formation Energy'),
        xaxis=dict( title = 'Spacegroup'))
iplot(go.Figure(data=data,layout=layout))    

### 空间群与形成能的关系

In [19]:
data = []
for spacegroup in df['spacegroup'].value_counts().index.tolist():
    y0 = df[df['spacegroup']==spacegroup]['bandgap_energy_ev'].values
    #y0 = np.log(y0+1)
    data.append(go.Box(y=y0, name=str(spacegroup), boxpoints = 'suspectedoutliers',boxmean='sd'))
    layout = go.Layout(
        title = "Spacegroup vs. Bandgap Energy",
        yaxis=dict( title = 'Bandgap Energy'),
        xaxis=dict( title = 'Spacegroup'))
iplot(go.Figure(data=data,layout=layout))